### Audiobooks project - Machine Learning part with hyperparameters tuning

#### Problem

You are given data from an Audiobook App. Logically, it relates to the audio versions of books ONLY. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.

The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertising to him/her. If we can focus our efforts SOLELY on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.

You have a .csv summarizing the data. There are several variables: Customer ID, ), Book length overall (sum of the minute length of all purchases), Book length avg (average length in minutes of all purchases), Price paid_overall (sum of all purchases) ,Price Paid avg (average of all purchases), Review (a Boolean variable whether the customer left a review), Review out of 10 (if the customer left a review, his/her review out of 10, Total minutes listened, Completion (from 0 to 1), Support requests (number of support requests; everything from forgotten password to assistance for using the App), and Last visited minus purchase date (in days).

These are the inputs (excluding customer ID, as it is completely arbitrary. It's more like a name, than a number).

The targets are a Boolean variable (0 or 1). We are taking a period of 2 years in our inputs, and the next 6 months as targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've gone to a competitor or didn't like the Audiobook way of digesting information. 

The task is simple: create a machine learning algorithm, which is able to predict if a customer will buy again. 

This is a classification problem with two classes: won't buy and will buy, represented by 0s and 1s. 

In [1]:
# Importing libraries
import numpy as np
import tensorflow as tf
import keras_tuner as kt

In [2]:
# Loading the data
npz = np.load('data/audiobooks_data_train_v1.npz')
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('data/audiobooks_data_validation_v1.npz')
#We have to make sure that all inputs are floats and targets are int so we add .astype() to every each of them
validation_inputs = npz['inputs'].astype(float)
validation_targets = npz['targets'].astype(int)

npz = np.load('data/audiobooks_data_test_v1.npz')
test_inputs, test_targets = npz['inputs'].astype(float), npz['targets'].astype(int)

### NN Model

### The model with the hyperparameters tuning using keras library keras_tuner

In [6]:
input_size = 10 # 10 predictors
output_size = 2 # output is 0 or 1

def model_builder(hp):
    hp_units = hp.Int('units', min_value=50, max_value=500, step=50)
    model = tf.keras.Sequential([
                             tf.keras.layers.Dense(units=hp_units, activation='relu'),
                             tf.keras.layers.Dense(units=hp_units, activation='relu'),
                             tf.keras.layers.Dense(units=hp_units, activation='softmax')
                            ])
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate), 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    return model

In [7]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs = 100,
                     factor = 3,
                     directory = 'data/',
                     project_name = 'audio_hp_tune'
                    )

In [8]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience=2)

In [10]:
tuner.search(train_inputs,
             train_targets,
             epochs = 50,
             callbacks=[early_stopping],
             validation_data=(validation_inputs, validation_targets),
             verbose=2)

best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 28 Complete [00h 00m 02s]
val_accuracy: 0.8165547847747803

Best val_accuracy So Far: 0.8165547847747803
Total elapsed time: 00h 01m 09s
INFO:tensorflow:Oracle triggered exit

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 300 and the optimal learning rate for the optimizer
is 0.01.



In [ ]:
# model.fit(train_inputs,
#           train_targets,
#           batch_size = batch_size,
#           epochs = max_epochs,
#           callbacks=[early_stopping],
#           validation_data=(validation_inputs, validation_targets),
#           verbose=2)

### Testing the model

In [7]:
#test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3563 - accuracy: 0.8103


In [8]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.36. Test accuracy: 81.03%
